In [1]:
import gym

In [4]:
env = gym.make("Taxi-v3").env
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [5]:
env.reset()

7

In [7]:
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [9]:
env.reset()
env.render()
print("Action Space{}".format(env.action_space))
print("Action Space{}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action SpaceDiscrete(6)
Action SpaceDiscrete(500)


In [12]:
state = env.encode(3,1,2,0)
print("State :", state)
env.render()

State : 328
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [13]:
env.P[328]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

In [17]:
env.s = 328
epochs = 0
penalties, reward = 0,0
frames= []
done = False
while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)
    if reward == -10:
        penalties += 1
        
    frames.append({   
    'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
    }
    )
    epochs +=1
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))
    


Timesteps taken: 1079
Penalties incurred: 354


In [25]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
#         print(frame['frame'].getvalue())
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
        
print_frames(frames)

Timestep: 1079
State: 0
Action: 5
Reward: 20


In [27]:
import numpy  as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [29]:
""" Training The Agent """
import random
from IPython.display import clear_output

alpha = 0.1
gamma = 0.6
epsilon = 0.1
all_epochs = []
all_penalties = []
for i in range(1, 100001):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0,1) < epsilon:
            action = env.action_space.sample()  #Explore action space
        else:
            action = np.argmax(q_table[state]) #Exploit learned values
        
        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")



Episode: 100000
Training finished.



In [30]:
q_table[328]

array([ -2.40943541,  -2.27325184,  -2.41396927,  -2.36299859,
       -10.52639717, -10.68579624])

In [32]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 500

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 500 episodes:
Average timesteps per episode: 12.906
Average penalties per episode: 0.0
